### Machine learning: Model forecasting
##### Business goal: Predict future temperature using an autoregressive model
##### Data set: ECAD temperature data from Berlin-Tempelhof

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
from statsmodels.tsa import stattools
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from sklearn.linear_model import LinearRegression
plt.rcParams['figure.figsize'] = (12, 6)

In [20]:
# import data as df
df =  pd.read_csv('../data/temp_modified.csv', index_col=0, parse_dates=True)

In [21]:
df['year']= df.index.year
df['timestep'] = range(len(df)) # add column named "timestep" with the lenght of the dataframe df
df

In [23]:
df.index.month

Int64Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            ...
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
           dtype='int64', name='date', length=53722)

In [24]:
# convert categorical variables into dummy/indicator variables
seasonal_dummies = pd.get_dummies(df.index.month, prefix='month', drop_first=True).set_index(df.index)
df = df.join(seasonal_dummies)

In [26]:
X = df.drop(['temperature'], axis=1)
y = df['temperature']

In [27]:
m = LinearRegression()
m.fit(X, y)

LinearRegression()

In [28]:
# calculate remainder
df['trend_seasonal'] = m.predict(X)
df['differences'] = df['temperature'] - df['trend_seasonal']

## Using autoregressive model


In [30]:

selected_order = ar_select_order(df['differences'], maxlag=12)
# finding the number of lags to use
selected_order.ar_lags
# estimate prediction error using AIC
selected_order.aic

/Users/lisakalfhues/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/lisakalfhues/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


{(1, 2, 3, 4, 5): 242161.15925989382,
 (1, 2, 3, 4, 5, 6, 7): 242161.40850547323,
 (1, 2, 3, 4, 5, 6): 242161.74336996768,
 (1, 2, 3, 4, 5, 6, 7, 8): 242161.9265241778,
 (1, 2, 3, 4, 5, 6, 7, 8, 9): 242163.69160680243,
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10): 242165.61588013096,
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11): 242167.2179981555,
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12): 242169.1976950869,
 (1, 2, 3, 4): 242169.98511948815,
 (1, 2, 3): 242182.20441693725,
 (1, 2): 242445.22963903632,
 (1,): 243408.6383739422,
 0: 299235.46536032454}

In [31]:
ar_model = AutoReg(endog=df['differences'], lags=4, trend="n").fit()
ar_model.summary()

/Users/lisakalfhues/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:            differences   No. Observations:                53722
Model:                     AutoReg(4)   Log Likelihood             -121099.803
Method:               Conditional MLE   S.D. of innovations              2.306
Date:                Mon, 10 Apr 2023   AIC                         242209.605
Time:                        14:00:12   BIC                         242254.063
Sample:                    01-05-1876   HQIC                        242223.485
                         - 01-31-2023                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
differences.L1     0.9196      0.004    213.178      0.000       0.911       0.928
differences.L2    -0.1943      0.006    -33.178      0.000      -0.206      -0.183
differences.L3     0.0552      0.006      9.433      0.000       0.044       0.067
differences.L4     0.0163      0.004      3.770      0.000       0.008       0.025
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.2579           -0.0000j            1.2579           -0.0000
AR.2            1.0066           -2.5135j            2.7076           -0.1894
AR.3            1.0066           +2.5135j            2.7076            0.1894
AR.4           -6.6673           -0.0000j            6.6673           -0.5000
-----------------------------------------------------------------------------
"""

In [32]:
# weather forcast
ar_model.forecast(steps=3)

/Users/lisakalfhues/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


2023-02-01    2.556548
2023-02-02    1.854268
2023-02-03    1.442957
Freq: D, dtype: float64